# 0. Set-up

In [1]:
## Import packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = (20, 7)

In [3]:
## Set paths
user = os.getenv('USERNAME')
shared_workspace = '/home/mdawkins/modelling_club'
user_dir = os.path.join(shared_workspace, user)
data_dir = os.path.join(shared_workspace, 'raw_data')
data_output_dir = os.path.join(shared_workspace, 'raw_data_lfs/engineered/application/')

# 2. Import data

In [4]:
home_loan_train = pd.read_csv(data_dir + '/raw/application_train.csv',index_col='SK_ID_CURR')
train_response = home_loan_train['TARGET']

home_loan_test = pd.read_csv(data_dir + '/raw/application_test_noTarget.csv',index_col='SK_ID_CURR')
home_loan_test.loc[:, 'TARGET'] = None

application = pd.concat([home_loan_train,home_loan_test],axis=0)

# Macro

In [5]:
application["LOAN_LENGTH"] = application["AMT_CREDIT"]/application["AMT_ANNUITY"]

application["LOAN_STRESS"] = application["AMT_ANNUITY"]/application["AMT_INCOME_TOTAL"]

application["PRICE_CREDIT_RATIO"] = application["AMT_GOODS_PRICE"]/application["AMT_CREDIT"]
application["PRICE_CREDIT_RATIO"].loc[application["PRICE_CREDIT_RATIO"]>1.25] = 1.25
application["PRICE_CREDIT_RATIO"].loc[application["PRICE_CREDIT_RATIO"]<0.5] = 0.5

application["DOWN_PAYMENT"] = application["AMT_GOODS_PRICE"]-application["AMT_CREDIT"]

application["CREDIT_IN_YEARS_INCOME"] = application["AMT_CREDIT"]/application["AMT_INCOME_TOTAL"]

application["INCOME_PER_FAMILY_MEMBER"] = application["AMT_INCOME_TOTAL"]/application["CNT_FAM_MEMBERS"]

/home/mdawkins/jupyter/conda/envs/default/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
application["AMT_ANNUITY"].loc[application["AMT_ANNUITY"]>100000] = 100000

application["AMT_CREDIT"].loc[application["AMT_CREDIT"]>2000000] = 2000000

application["AMT_GOODS_PRICE"].loc[application["AMT_GOODS_PRICE"]>2000000] = 2000000

application["AMT_INCOME_TOTAL"].loc[application["AMT_INCOME_TOTAL"]>1000000] = 1000000

application["LOAN_LENGTH"].loc[application["LOAN_LENGTH"]>40] = 40

application["LOAN_STRESS"].loc[application["LOAN_STRESS"]>0.75] = 0.75



# Demographic

In [7]:
application["DAYS_EMPLOYED"].loc[application["DAYS_EMPLOYED"]==365243.000000] = None

In [8]:
application["OCCUPATION_TYPE"].loc[application["OCCUPATION_TYPE"]=="Low-skill Laborers"] = "Laborers"
application["OCCUPATION_TYPE"].loc[application["OCCUPATION_TYPE"]=="Waiters/barmen staff"] = "Laborers"
application["OCCUPATION_TYPE"].loc[application["OCCUPATION_TYPE"]=="IT staff"] = "High skill tech staff"
application["OCCUPATION_TYPE"].loc[application["OCCUPATION_TYPE"]=="HR staff"] = "Secretaries"

In [9]:
application['DAYS_EMPLOYED_PERCENT'] = application['DAYS_EMPLOYED'] / application['DAYS_BIRTH']
application["YEARS_EMPLOYED"] = np.floor(application["DAYS_EMPLOYED"]/365)
application["YEARS_BIRTH"] = np.floor(application["DAYS_BIRTH"]/365)
application["HIGHER_ED"] = application["NAME_EDUCATION_TYPE"].isin(['Higher education', 'Academic degree'])

In [10]:
application["NAME_INCOME_TYPE"].loc[application["NAME_INCOME_TYPE"]=="Student"] = "Unemployed"
application["NAME_INCOME_TYPE"].loc[application["NAME_INCOME_TYPE"]=="Maternity leave"] = "Unemployed"
application["NAME_INCOME_TYPE"].loc[application["NAME_INCOME_TYPE"]=="Pensioner"] ="Unemployed"
application["NAME_INCOME_TYPE"].loc[application["NAME_INCOME_TYPE"]=="Businessman"] = "Working"

# External

NameError: name 'application' is not defined

# Regional

In [11]:
application["REGION_ID"] = application["REGION_POPULATION_RELATIVE"].astype("category")


In [ ]:
application = application.drop([ col for col in application.columns if "MODE" in col],axis=1)
application = application.drop([ col for col in application.columns if "MODE" in col],axis=1)

In [12]:
application

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,LOAN_STRESS,PRICE_CREDIT_RATIO,DOWN_PAYMENT,CREDIT_IN_YEARS_INCOME,INCOME_PER_FAMILY_MEMBER,DAYS_EMPLOYED_PERCENT,YEARS_EMPLOYED,YEARS_BIRTH,HIGHER_ED,REGION_ID
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
113072,0,Revolving loans,M,Y,Y,0,283500.0,180000.0,9000.0,180000.0,...,0.031746,1.000000,0.0,0.634921,141750.0,0.488376,-28.0,-56.0,False,0.005313
225465,0,Cash loans,F,N,Y,0,67500.0,182016.0,10291.5,144000.0,...,0.152467,0.791139,-38016.0,2.696533,33750.0,NaN,NaN,-67.0,False,0.020713
260656,0,Cash loans,F,N,Y,0,112500.0,315000.0,17716.5,315000.0,...,0.157480,1.000000,0.0,2.800000,56250.0,0.621163,-34.0,-54.0,False,0.007020
445761,0,Cash loans,F,N,Y,0,675000.0,2000000.0,59301.0,1800000.0,...,0.087853,0.834725,-356400.0,3.194667,337500.0,0.073525,-4.0,-45.0,False,0.022800
421144,0,Cash loans,F,N,Y,0,180000.0,679500.0,19867.5,679500.0,...,0.110375,1.000000,0.0,3.775000,90000.0,0.017771,-1.0,-29.0,False,0.006008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456245,None,Cash loans,F,N,Y,3,81000.0,269550.0,11871.0,225000.0,...,0.146556,0.834725,-44550.0,3.327778,16200.0,0.080704,-3.0,-36.0,False,0.009175
456247,None,Cash loans,F,N,Y,0,112500.0,345510.0,17770.5,247500.0,...,0.157960,0.716332,-98010.0,3.071200,112500.0,0.033614,-2.0,-33.0,True,0.022800
456248,None,Cash loans,F,N,Y,0,153000.0,331920.0,16096.5,225000.0,...,0.105206,0.677874,-106920.0,2.169412,153000.0,0.434481,-20.0,-46.0,False,0.022625


# Flag Documnets

In [6]:
application = application.drop([ col for col in application.columns if "FLAG_DOCUMENT" in col],axis=1)

# Other 

In [7]:
application = application.drop(["WEEKDAY_APPR_PROCESS_START","HOUR_APPR_PROCESS_START"],axis=1)

['APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'TOTALAREA_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

# Write file

In [13]:
application.to_pickle(os.path.join(data_output_dir,'application.pkl'))
